<a href="https://colab.research.google.com/github/sarraj-sareef/metaphor-detection-tamil/blob/main/Binary_classifier_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets evaluate accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.2 MB/s eta 0:00:00


In [ ]:
# Data processing
import pandas as pd
import numpy as np

# Modeling
import tensorflow as tf
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback, TextClassificationPipeline

# Hugging Face Dataset
from datasets import Dataset

# Model performance evaluation
import evaluate
# from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

In [ ]:
from google.colab import drive

drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# def compute_metrics(p):
#     print(type(p))
#     pred, labels = p
#     pred = np.argmax(pred, axis=1)

#     accuracy = accuracy_score(y_true=labels, y_pred=pred)
#     recall = recall_score(y_true=labels, y_pred=pred)
#     precision = precision_score(y_true=labels, y_pred=pred)
#     f1 = f1_score(y_true=labels, y_pred=pred)

#     return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [ ]:
# def tokenize_function(examples):
#     return tokenizer(examples["Lyrics"], padding="max_length", truncation=True)

In [ ]:

df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/binary_classification.csv')
df.head()

,Song Name,Composer,Album,Lyricist,text,labels
0,அந்த கண்ண பார்த்தாக்கா,அனிருத் ரவிசந்தர்,மாஸ்டர்,விக்னேஷ் சிவன்,அந்த கண்ண பார்த்தாக்கா லவ்வு தானா தோனாதா,0
1,அந்த கண்ண பார்த்தாக்கா,அனிருத் ரவிசந்தர்,மாஸ்டர்,விக்னேஷ் சிவன்,அவன் கிட்ட போனாக்கா மனம் மானா மாறாதா,1
2,அந்த கண்ண பார்த்தாக்கா,அனிருத் ரவிசந்தர்,மாஸ்டர்,விக்னேஷ் சிவன்,அகமெல்லாம் அவன்தான் அவன்தான் இருந்தான்,0
3,அந்த கண்ண பார்த்தாக்கா,அனிருத் ரவிசந்தர்,மாஸ்டர்,விக்னேஷ் சிவன்,நடந்தால் அவன் கனவெல்லாமே அவன் முகம் தானே…ஏ….,0
4,அந்த கண்ண பார்த்தாக்கா,அனிருத் ரவிசந்தர்,மாஸ்டர்,விக்னேஷ் சிவன்,அழகன்தான் அவன்தான் அவன்தான் அழகா அளவா அவன் சிர...,0


In [ ]:
df.drop(['Song Name', 'Composer', 'Album', 'Lyricist'], axis=1, inplace=True)

df.head()

,text,labels
0,அந்த கண்ண பார்த்தாக்கா லவ்வு தானா தோனாதா,0
1,அவன் கிட்ட போனாக்கா மனம் மானா மாறாதா,1
2,அகமெல்லாம் அவன்தான் அவன்தான் இருந்தான்,0
3,நடந்தால் அவன் கனவெல்லாமே அவன் முகம் தானே…ஏ….,0
4,அழகன்தான் அவன்தான் அவன்தான் அழகா அளவா அவன் சிர...,0


In [ ]:
X=list(df['text'])

In [ ]:
y=list(df['labels'])

In [ ]:
# Training dataset
train_data = df.sample(frac=0.8, random_state=42)

# Testing dataset
test_data = df.drop(train_data.index)
print(f'The training dataset has {len(train_data)} records.')
print(f'The testing dataset has {len(test_data)} records.')

The training dataset has 741 records.
The testing dataset has 185 records.


In [ ]:
hg_train_data = Dataset.from_pandas(train_data)
hg_test_data = Dataset.from_pandas(test_data)

In [ ]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("monsoon-nlp/tamillion")
# tokenizer = AutoTokenizer.from_pretrained("google/muril-base-cased")
# tokenized_data = tokenizer(X_train, return_tensors="np", padding=True)
# # Tokenizer returns a BatchEncoding, but we convert that to a dict for Keras
# tokenized_data = dict(tokenized_data)

# labels = np.array(y_train)

tokenizer_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/736 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/837k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
def tokenize_dataset(data):
    return tokenizer(data["text"],
                     max_length=32,
                     truncation=True,
                     padding="max_length")

# Tokenize the dataset
dataset_train = hg_train_data.map(tokenize_dataset)
dataset_test = hg_test_data.map(tokenize_dataset)

Map:   0%|          | 0/741 [00:00<?, ? examples/s]

Map:   0%|          | 0/185 [00:00<?, ? examples/s]

In [ ]:
print(dataset_train)
print(dataset_test)

Dataset({
    features: ['text', 'labels', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 741
})
Dataset({
    features: ['text', 'labels', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 185
})


In [ ]:
from transformers import AutoModelForSequenceClassification
# from tensorflow.keras.optimizers import Adam

# Load and compile our model
model = AutoModelForSequenceClassification.from_pretrained("monsoon-nlp/tamillion",num_labels=2)
# Lower learning rates are often better for fine-tuning transformers
# model.compile(optimizer=Adam(3e-5))  # No loss argument!

# model.fit(tokenized_data, labels)

model.safetensors:   0%|          | 0.00/467M [00:00<?, ?B/s]

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monsoon-nlp/tamillion and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir="./metaphor_detection/",
    num_train_epochs=2,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    eval_steps = 0
)

In [ ]:
def compute_metrics(eval_pred):
    metric = evaluate.load("accuracy")
    logits, labels = eval_pred
    # probabilities = tf.nn.softmax(logits)
    predictions = np.argmax(logits, axis=1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_test,
    compute_metrics=compute_metrics,
    # callbacks=[EarlyStoppingCallback(early_stopping_patience=1)]
)

trainer.train()

Step,Training Loss
10,0.700600
20,0.705000
30,0.695400
40,0.698700
50,0.711600
60,0.708700
70,0.693800
80,0.707100
90,0.702300
100,0.695400


TrainOutput(global_step=186, training_loss=0.6995170372788624, metrics={'train_runtime': 18.0269, 'train_samples_per_second': 82.21, 'train_steps_per_second': 10.318, 'total_flos': 24370661502720.0, 'train_loss': 0.6995170372788624, 'epoch': 2.0})

In [ ]:
y_test_predict = trainer.predict(dataset_test)

# Take a look at the predictions
y_test_predict

PredictionOutput(predictions=array([[-0.01171095,  0.01977688],
       [-0.01171097,  0.01977687],
       [-0.01171095,  0.01977686],
       [-0.01171093,  0.01977688],
       [-0.01171097,  0.01977686],
       [-0.01171095,  0.01977687],
       [-0.01171094,  0.01977687],
       [-0.01171096,  0.01977685],
       [-0.01171092,  0.01977689],
       [-0.01171095,  0.01977686],
       [-0.01171098,  0.01977687],
       [-0.01171096,  0.01977685],
       [-0.01171094,  0.01977688],
       [-0.01171094,  0.01977685],
       [-0.01171093,  0.01977686],
       [-0.01171094,  0.01977687],
       [-0.01171094,  0.01977686],
       [-0.01171096,  0.01977688],
       [-0.01171093,  0.0197769 ],
       [-0.01171092,  0.01977688],
       [-0.01171094,  0.01977686],
       [-0.01171095,  0.01977686],
       [-0.01171095,  0.01977686],
       [-0.01171093,  0.01977689],
       [-0.01171095,  0.01977685],
       [-0.01171095,  0.01977686],
       [-0.01171095,  0.01977688],
       [-0.01171095,  0.01

In [ ]:
y_test_logits = y_test_predict.predictions

# First 5 predicted probabilities
y_test_logits[:5]

array([[-0.01171095,  0.01977688],
       [-0.01171097,  0.01977687],
       [-0.01171095,  0.01977686],
       [-0.01171093,  0.01977688],
       [-0.01171097,  0.01977686]], dtype=float32)

In [ ]:
# Predicted probabilities
y_test_probabilities = tf.nn.softmax(y_test_logits)

# First 5 predicted logits
y_test_probabilities[:5]

<tf.Tensor: shape=(5, 2), dtype=float32, numpy=
array([[0.49212873, 0.5078713 ],
       [0.49212873, 0.5078713 ],
       [0.49212873, 0.5078713 ],
       [0.49212873, 0.5078713 ],
       [0.49212873, 0.5078713 ]], dtype=float32)>

In [ ]:
# Predicted labels
y_test_pred_labels = np.argmax(y_test_probabilities, axis=1)

# First 5 predicted probabilities
y_test_pred_labels[:5]

array([1, 1, 1, 1, 1])

In [ ]:
# Actual labels
y_test_actual_labels = y_test_predict.label_ids

# First 5 predicted probabilities
y_test_actual_labels[:5]

array([1, 0, 0, 1, 0])

In [ ]:
trainer.evaluate(dataset_test)

{'eval_loss': 0.6952285170555115,
 'eval_accuracy': 0.43783783783783786,
 'eval_runtime': 1.0351,
 'eval_samples_per_second': 178.73,
 'eval_steps_per_second': 11.593,
 'epoch': 2.0}

In [ ]:
# Load f1 metric
metric_f1 = evaluate.load("f1")

# Compute f1 metric
metric_f1.compute(predictions=y_test_pred_labels, references=y_test_actual_labels)

{'f1': 0.6090225563909775}

In [ ]:
# Load recall metric
metric_recall = evaluate.load("recall")

# Compute recall metric
metric_recall.compute(predictions=y_test_pred_labels, references=y_test_actual_labels)

{'recall': 1.0}